In [1]:
# Packages to Import
from threading import Thread
import numpy as np
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
import csv
from itertools import combinations
import scipy.io as sio
from scipy import spatial

In [2]:
class ThreadWithReturnValue(Thread):
    """ 
    Created a Thread subclass. It is a workable doaround,
    but it accesses "private" data structures that are specific to Thread implementation, so 
    things will get a little hairy.
    """
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        
        """ Runs the function in a specified for the thread. """
        # If the target function is specified
        if self._target is not None:
            
            # Run the function 
            self._return = self._target(*self._args,
                                                **self._kwargs)
            
    def join(self, *args):
        """ Returns the value of target function running in the thread. """
        
        Thread.join(self, *args)
        return self._return

In [3]:
def getData(source_name):
    """ Returns the data depending on the cycle number related and the specific sound source.
    
        Keyword arugments:
        
        source_name -- specifies each sound cycle 
    
    """

    # Create a dictionary with all cycles: Regular S1 and S1 (top) Recovered S1 and S2 (bottom)
    source_name_dict = {f'S{x}_Cycle{y}': [f'S{x}/S{x}_Cycle{y}', f'S{x}'] for x in range(1,3) for y in range(24)} 
    #source_name_dict = {f'S{x}_Cycle{y}': [f'Recovered_S{x}/S{x}_Cycle{y}', f'S{x}'] for x in range(1,3) for y in range(24)}
    
    # Match the correct data with the name
    for key in source_name_dict.keys():
        if source_name == key:
            data = sio.loadmat(source_name_dict[key][0])
            sound_data = data[source_name_dict[key][1]]
    
    # Return the sound data
    return sound_data

In [4]:
def centroid(*args):  
    
    """ Returns the center of n number of microphones. 
    
    Keyword Arguments:
    
    args -- location of each n microphone 
    
    """

    # Initiate 
    microphone_array = np.zeros((len(args), len(args[1])))
    
    # Converts micrphone locations into an array
    for i in range(len(args)):
        microphone_array[i,:] = np.array(args[i])
   
    # Finds the centroid
    return np.sum(microphone_array, axis=0)/len(args)

In [5]:
def mic_run(data, *args):
    
    """ Returns each of the n microphone locations and the signals list corresponding to the specific microphone.
        Note: The microphone locations are under a new coordinate system in relation to the center of the box
              (whose center = [(0.34925/2),(0.219964/2),(0.2413/2)] is the origin)
    
        Keyword arguments:
            data -- the signal associated with each microphone
            args -- list of the microphones 
    
    """
    
    # Empty Lists
    signal_list = []
    mic_location = []
    
    # Dictionary of the microphone locations and their respective signals
    # Note: order is #mic number (from 1 -12), followed by location of channel (to get actual signal)
    microphones_locations_dict = dict({
        'mic1': [[-0.102235, -0.109982, 0.056388], data[0]],  
        'mic2': [[-0.102235, -0.109982, 0.001524],  data[1]],
        'mic3': [[-0.102235, -0.109982, -0.053340], data[2]], 
        'mic4': [[-0.102235, -0.109982, -0.108204], data[3]],
        'mic5': [[-0.052197, -0.109982, 0.056388], data[4]],
        'mic6': [[-0.052197, -0.109982, 0.001524], data[5]],
        'mic7': [[-0.052197, -0.109982, -0.053340], data[6]],
        'mic8': [[-0.052197, -0.109982, -0.108204], data[7]],
        'mic9': [[-0.027304, -0.109982, 0.056388], data[8]],
        'mic10': [[-0.027304, -0.109982, 0.001524], data[9]],
        'mic11': [[-0.027304, -0.109982, -0.053340], data[10]],
        'mic12': [[-0.027304, -0.109982, -0.108204], data[11]]
        }) 

    # Look for a match between the dictionary of microphone locations and the microphone in the list
    for arg in args:
        for key in microphones_locations_dict.keys():
            if arg == key:
                # Record the location
                mic_location.append(microphones_locations_dict[key][0])
                
                # Record the signal
                signal_list.append(microphones_locations_dict[key][1])
    
   # Return the whole signal list as well all the specific microphone locations
    return signal_list, mic_location

In [6]:
def difference_of_arrivals(speed_sound,signal_list,algo_name, *mic_location):

    """ Returns an azimuth and co-latitude for each pair of microphones. 

        Keyword arugments:

            sound_speed -- Specific speed of sound
            signal_list -- the microphone signals
            algo_name -- Specific distance of arrival (DOA) method
            mic_location -- location of each microphone
    """
    
    #Constants 
    fs = 16000  # sampling frequency
    nfft = 256  # FFT size
    
    # Add 3-microphone array in [x,y,z] order
    R = np.vstack(list(zip(*mic_location)))
    
    # Create an array of a short fourier transformed frequency signal
    X = np.array([pra.stft(signal, nfft, nfft//2,transform=np.fft.rfft).T for signal in signal_list])
    
    # Frequency Range
    freq_range = [0,250]

    # Construct the new DOA object
    doa = pra.doa.algorithms[algo_name](L=R, fs=fs, nfft=nfft, c=sound_speed, num_src=1, max_four=4,
    dim=3,azimuth=np.linspace(-180.,180.,360)*np.pi/180,
    colatitude=np.linspace(-90.,90.,180)*np.pi/180)
    
    # Locate the sources
    doa.locate_sources(X, freq_range=freq_range)
    
    # Return all in radians
    return doa.azimuth_recon, doa.colatitude_recon

In [7]:
def main(sound_speed,algo_name,sound_data,combinations_number,S1_bool,source_name):
    
    """ Returns list of points that are either close to the point or the exact point itself.
    
        Keyword arguments:
        
            sound_speed -- specific speed of sound
            algo_name -- specific distance of arrival (DOA) method to call
            sound_data -- specific data to perform the localizing
            combinations_number -- number of microphone to use
            S1_bool -- flag to indicate whether to find S1 or S2 sound source
            source_name -- indicates S1 or S2 and what specific cycle
    """
    
     # Optimization:
    if S1_bool:
        # List of specific microphones to quickly find S1
        mics = ['mic'+str(i) for i in [2,3,4,6,7,8]]

    else:
        # List of specific microphones to quickly find S2
        mics = ['mic'+str(i) for i in range(1,13)]
        
        #mics = ['mic'+str(i) for i in [1,2,5,6,10,11]]
        #mic = ['mic'+str(i) for i in [2,3,6,7,10,11]]
        
        # Past MIC combinations
        #mics = ['mic'+str(i) for i in [1,2,5,6,10,11]] --> NADA
        #mics = ['mic'+str(i) for i in [1,2,3,5,6,7]] --> All cycles: NADA
        #mics = ['mic'+str(i) for i in [5,6,7,10,11]] --> NADA
    # TRASH
    #mics = ['mic'+str(i) for i in range(1,13)]
    
    # Creates a list of N microphone-combinations
    mic_list=list(combinations(mics,combinations_number))
    
    # number of mic pair splits to run 
    splits = len(mic_list)//5
    
    # Split up the mic list
    mic_split_list = [mic_list[i*splits:(i+1)*splits] for i in range((len(mic_list)+splits-1)//splits)]
    
    # Store the final output
    outputs_list = []

    # Constants
    tol = 3e-3
    r = np.arange(0,0.5,tol)[:, np.newaxis]

    for j in range(splits):
              
        # Pick the signal array and the associated pair of n microphone combinations
        # Note: Multi-threaded the mic_run function for faster use
        twrv1 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[0][j]))
        twrv2 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[1][j]))
        twrv3 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[2][j]))
        twrv4 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[3][j]))
        twrv5 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[4][j]))
        #twrv6 = ThreadWithReturnValue(target=mic_run, args=(sound_data, *mic_split_list[5][j]))

        twrv1.start()
        twrv2.start()
        twrv3.start()
        twrv4.start()
        twrv5.start()
        #twrv6.start()
        
        # Return signal and microphone locations
        [signal_1, mic_locations_1] = twrv1.join()
        [signal_2, mic_locations_2] = twrv2.join()
        [signal_3, mic_locations_3] = twrv3.join()
        [signal_4, mic_locations_4] = twrv4.join()
        [signal_5, mic_locations_5] = twrv5.join()
        #[signal_6, mic_locations_6] = twrv6.join()
   
        # Find the centriods of the pairs of n microphone combinations
        twrv7 = ThreadWithReturnValue(target=centroid, args=(mic_locations_1))
        twrv8 = ThreadWithReturnValue(target=centroid, args=(mic_locations_2))
        twrv9 = ThreadWithReturnValue(target=centroid, args=(mic_locations_3))
        twrv10 = ThreadWithReturnValue(target=centroid, args=(mic_locations_4))
        twrv11 = ThreadWithReturnValue(target=centroid, args=(mic_locations_5))
        #twrv12 = ThreadWithReturnValue(target=centroid, args=(mic_locations_6))

        twrv7.start()
        twrv8.start()
        twrv9.start()
        twrv10.start()
        twrv11.start()
        #twrv12.start()
        
        # Return the centeriods
        centroid_1 = twrv7.join()
        centroid_2 = twrv8.join()
        centroid_3 = twrv9.join()
        centroid_4 = twrv10.join()
        centroid_5 = twrv11.join()
        #centroid_6 = twrv12.join()
        
        # Perform the distance of arrival methods to find closest azimuth and colatitude angles
        twrv13 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_1,algo_name,*mic_locations_1))
        twrv14 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_2,algo_name,*mic_locations_2))
        twrv15 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_3,algo_name,*mic_locations_3))
        twrv16 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_4,algo_name,*mic_locations_4))
        twrv17 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_5,algo_name,*mic_locations_5))
        #twrv18 = ThreadWithReturnValue(target = difference_of_arrivals, args=(sound_speed,signal_6,algo_name,*mic_locations_6))
        
        twrv13.start()
        twrv14.start()
        twrv15.start()
        twrv16.start()
        twrv17.start()
        #twrv18.start()
        
        # Desired angles 
        azimuth_recon_1, colatitude_recon_1 = twrv13.join()
        azimuth_recon_2, colatitude_recon_2 = twrv14.join()
        azimuth_recon_3, colatitude_recon_3 = twrv15.join()
        azimuth_recon_4, colatitude_recon_4 = twrv16.join()
        azimuth_recon_5, colatitude_recon_5 = twrv17.join()
        #azimuth_recon_6, colatitude_recon_6 = twrv18.join()
        
        # Desired cartesian coordinates
        cartesian_1 = np.array([np.cos(azimuth_recon_1)*np.sin(colatitude_recon_1),np.sin(azimuth_recon_1)*np.sin(colatitude_recon_1),np.cos(colatitude_recon_1)])
        cartesian_2 = np.array([np.cos(azimuth_recon_2)*np.sin(colatitude_recon_2),np.sin(azimuth_recon_2)*np.sin(colatitude_recon_2),np.cos(colatitude_recon_2)])
        cartesian_3 = np.array([np.cos(azimuth_recon_3)*np.sin(colatitude_recon_3),np.sin(azimuth_recon_3)*np.sin(colatitude_recon_3),np.cos(colatitude_recon_3)])
        cartesian_4 = np.array([np.cos(azimuth_recon_4)*np.sin(colatitude_recon_4),np.sin(azimuth_recon_4)*np.sin(colatitude_recon_4),np.cos(colatitude_recon_4)])
        cartesian_5 = np.array([np.cos(azimuth_recon_5)*np.sin(colatitude_recon_5),np.sin(azimuth_recon_5)*np.sin(colatitude_recon_5),np.cos(colatitude_recon_5)])
        #cartesian_6 = np.array([np.cos(azimuth_recon_6)*np.sin(colatitude_recon_6),np.sin(azimuth_recon_6)*np.sin(colatitude_recon_6),np.cos(colatitude_recon_6)])
        
        # Re-center them via adding the centroid
        estimate_1 = r*cartesian_1.T + np.array(centroid_1)[np.newaxis,:] 
        estimate_2 = r*cartesian_2.T + np.array(centroid_2)[np.newaxis,:]
        estimate_3 = r*cartesian_3.T + np.array(centroid_3)[np.newaxis,:] 
        estimate_4 = r*cartesian_4.T + np.array(centroid_4)[np.newaxis,:]
        estimate_5 = r*cartesian_5.T + np.array(centroid_5)[np.newaxis,:] 
        #estimate_6 = r*cartesian_6.T + np.array(centroid_6)[np.newaxis,:]
        
        # Add to an output list
        outputs_list.extend((estimate_1, estimate_2, estimate_3, estimate_4, estimate_5)) #, estimate_6))
    
    # Make a numpy array of them 
    all_estimates = np.array(outputs_list)
    
    # Reshape them to (_, 3) which is proper format for the tree
    total_array = np.reshape(all_estimates,(all_estimates.shape[0]*all_estimates.shape[1], all_estimates.shape[2]))
    
    # Put the whole list into a tree data data structure
    tree = spatial.KDTree(total_array)
    
    if S1_bool:
        # Find the points closest to where S1 is
        indices_S1 = tree.query_ball_point([-0.110925,-0.003482,-0.06065], 2.5e-2)
        estimate = (total_array[indices_S1][j] for j in range(len(indices_S1)))
    else:
        # Find the points closest to where S2 is
        indices_S2 = tree.query_ball_point([-0.165225,-0.014282,-0.06565], 2.5e-2) 
        estimate = (total_array[indices_S2][j] for j in range(len(indices_S2)))
    
    # Write to a csv file to save the data
    filename = 'mic_'+str(combinations_number)+'_'+str(source_name)+'_sound_source_localization_c'+str(sound_speed)+'_'+str(algo_name)+'_KDTREE_shortcut_2_test_multithread.csv'
    with open(filename, mode='w') as sound_source_file:
        writer = csv.writer(sound_source_file,delimiter=',')

        # First Row of Data, names of the columns
        writer.writerow(['X Location', 'Y Location', 'Z Location'])

        # Write the rest of the results
        # Note they have not been converted back into correct x,y,z coordinates
        writer.writerows(estimate)

    sound_source_file.close()

In [ ]:
if __name__ == '__main__':
    """
    Performs the main script using all the sound data, specific speed of sound, 
    the names of the distance of arrival methods, and number of combinations of microphone pairs 
    """
    # Speed of sound
    sound_speed = 30
    
    # Number of Pairs of Microphone Combinations
    combinations_number = 3
    
    # Data: Number of Cycles for each Sound Source
    cycles = ['Cycle'+str(i) for i in range(3,24)] 
    soundSources = ['S'+str(i) for i in range(2,3)] 
    sound_list = [soundSource+'_'+cycle for soundSource in soundSources for cycle in cycles]
    
    #DEBUG:
    S1_bool = False
    
#     # When to find S1 and S2
#     S1_bool = True
    
    # Now test all the algorithms available
    algo_names = ['SRP','MUSIC','TOPS']
    
    for source_name in sound_list:
        for algo_name in algo_names:
            
#             # Check if name is S2
#             if source_name in ['S'+str(i)+'_'+'Cycle'+str(j) for i in range(2,3) for j in range(24)]:
#                 S1_bool = False
            
            # Get the Sound Data
            sound_data = getData(source_name)

            # Get the final results
            main(sound_speed,algo_name,sound_data,combinations_number,S1_bool,source_name)